# ETL Project

#Goal 
To extract weather data from NOCAA mainly Max Temp, Min Temp,Prcp from all weather stations in the country
and the stock prices of 3 major Agricultural Companies(Namely Mosaic Company listed on NYSE:MOS,FMC Corp NYSE:FMC &
CF Industries listed on NYSE:CF


# Data Types Used
1.Zipped Tar File downloaded from NOCAA website containing binary(.dly) files of daily weather readings from all weather stations around the world

2.Used Pandas Data Reader to download stock prices from yahoo finance

3. Text Files containing station data(station id, Locations,Names)
4.Text file contain station inventory (used to identify stations that measure TMAX,TMIN and PRCP from selected dates

# Scope
Data scope was from 2015 to date

In [345]:
#IMPORT DEPENDENCIES
import pandas as pd
from sqlalchemy import create_engine
from pandas_datareader import data

## Imported Text files  i.e Station Info and Inventory and name cols


In [346]:
#iMPORT STATIONS.TEXT AND RENAME COLS
stations = pd.read_csv("ghcnd-stations.txt", delim_whitespace=True, names=["ID", "LAT", "LON", "ELEV", "ST", "NAME","GSN", "HCN", "WMOID"])

In [347]:
#COUNT STATIONS
stations.count()

ID       107098
LAT      107098
LON      107098
ELEV     107098
ST       107097
NAME      92194
GSN       68298
HCN       49810
WMOID     13418
dtype: int64

In [348]:
#iMPORT STATIONS INVENTORY AND RENAME COLS
inventory_stn = pd.read_csv("ghcnd-inventory.txt", delim_whitespace=True, names=["ID", "LAT", "LON", "ELEM" , "FIRST", "LAST"])

In [349]:
inventory_stn.head()

,ID,LAT,LON,ELEM,FIRST,LAST
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949


# #Selected Preffered weather elements i.e (TMAX,TMIN,PRCP ) and Stations Located in Inland United States

In [350]:
#SELECTED WEATHER ELEMENTS
us2 =inventory_stn[(inventory_stn.ELEM =="TMAX")|(inventory_stn.ELEM=="TMIN")|(inventory_stn.ELEM=="PRCP")]

In [351]:
us2.head()

,ID,LAT,LON,ELEM,FIRST,LAST
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
11,ACW00011647,17.1333,-61.7833,TMAX,1961,1961
12,ACW00011647,17.1333,-61.7833,TMIN,1961,1961


In [352]:
#FILTERED INVENTORY FOR YEARS FROM 2015
us3=us2[us2.FIRST>=2015]
us3.head()

,ID,LAT,LON,ELEM,FIRST,LAST
1167,ARM00087345,-31.4000,-64.1830,TMIN,2017,2017
1340,ARM00087823,-42.7330,-65.0670,TMIN,2017,2017
62934,BF1BI000001,25.7371,-79.2908,PRCP,2017,2018
62936,BF1CE000001,25.1552,-76.1768,PRCP,2016,2018
62938,BF1CE000005,25.1583,-76.1788,PRCP,2016,2017


In [353]:
#MERGERD STATIONS AND INVENTORY TO GET STATIONS THAT HAS DATA FOR PREFFERED ELEMENTS FOR THE SELECTED YEARS
Preffered_stns=pd.merge(stations,us3[["ID","ELEM","FIRST","LAST"]],how='inner',on=["ID"])
Preffered_stns['Country_code']=Preffered_stns['ID'].str[:2]
Preffered_stns=Preffered_stns[(Preffered_stns.Country_code =="US")&~Preffered_stns.ST.isin(["HI","UW","AK","PI"])]
Preffered_stns.head()

,ID,LAT,LON,ELEV,ST,NAME,GSN,HCN,WMOID,ELEM,FIRST,LAST,Country_code
603,US10buff039,40.8153,-98.8355,0.0,NE,GIBBON,4.7,N,NaN,PRCP,2015,2018,US
604,US10butl020,41.2557,-97.1301,0.0,NE,DAVID,CITY,0.2,N,PRCP,2015,2017,US
605,US10cass011,41.0289,-95.9874,0.0,NE,PLATTSMOUTH,3.1,WNW,NaN,PRCP,2015,2016,US
606,US10chas037,40.4152,-101.3746,0.0,NE,WAUNETA,0.3,SW,NaN,PRCP,2015,2018,US
607,US10chey060,41.1339,-102.9777,0.0,NE,SIDNEY,0.6,S,NaN,PRCP,2015,2018,US


# #Used OS and Tarfile Modules to import and Extract Binary Files

In [354]:
import os
import tarfile

def py_files(members):
    for tarinfo in members:
        if os.path.splitext(tarinfo.name)[1] == ".dly":
            yield tarinfo

tar = tarfile.open("ghcnd_all.tar.gz")
tar.extractall(members=py_files(tar))
tar.close()


# #Created a list to rename binary files

In [355]:
n=list(range(32))
n.remove(0)
n.append("E")
#CREATE AN EMPTY DATAFRAME
import numpy as np
# DI=pd.DataFrame(columns = ["year","month","element","id","Day","value"])


   # Imported All .dly files for GHCN folder
   # created an empty data frame to append tables
   # Created a For Loop that reads all the files in the folder and clean 
   # Used PathLib to identify file path
   # Used fortran file reader
   
   # Renamed Cols(i.e Daily Readings)
   # Extracted Station ID , year ,month, 
   # Melted Data
   # Appended Data from Preffered stations to Table DI

In [356]:
import pathlib

DI=pd.DataFrame(columns = ["year","month","element","id","Day","value"])
py = pathlib.Path().glob("ghcnd_all/*.dly")
for file in py:
    try:
        county_db=pd.read_fwf(file,colspecs='infer', widths=None,header=None)
        county_db['ELEM']=county_db[0]
        colscols2=county_db.columns.tolist()
# county_db 
        dictionary = dict(zip(colscols2, n))
        dat=county_db.rename(columns=dictionary )
        dat['state'] = dat['ELEM'].str.extract('([A-Z]\w{0,})', expand=True)
        dat['element']=dat['state'].str[-4:]
        dat['Country_code']=dat['state'].str[:2]
        dat['el3']=dat['state'].str[:-4]
        dat['year-month']=dat['el3'].str[-6:]
        dat['year']=dat['year-month'].str[:4]
        dat['month']=dat['year-month'].str[-2:]
        dat['id']=dat['state'].str[:11]
        df=dat[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,'year','month','element','id']]
        df=df[(df.id.isin(Preffered_stns.ID))]
        m = pd.melt(df, id_vars=['year','month','element','id'], var_name='Day')
        DI=DI.append(m,ignore_index=True)
    except:
        print(f'Bad Data Encountered Skipping to next')
  

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

...

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Encountered Skipping to next
Bad Data Enc

# Obtained DI with 8Million Rows

In [579]:
len(DI)

8439068

In [357]:
DataObtained=DI
DataObtained.head(20)

,year,month,element,id,Day,value
0,2015,09,PRCP,US10buff039,1,US10buff039201509PRCP-9999
1,2015,09,SNOW,US10buff039,1,US10buff039201509SNOW-9999
2,2015,10,PRCP,US10buff039,1,US10buff039201510PRCP 0
3,2015,10,SNOW,US10buff039,1,US10buff039201510SNOW-9999
4,2015,11,PRCP,US10buff039,1,US10buff039201511PRCP 0
5,2015,11,SNOW,US10buff039,1,US10buff039201511SNOW 0
6,2015,12,PRCP,US10buff039,1,US10buff039201512PRCP 13
7,2015,12,SNOW,US10buff039,1,US10buff039201512SNOW 0
8,2016,01,PRCP,US10buff039,1,US10buff039201601PRCP 0
9,2016,01,SNOW,US10buff039,1,US10buff039201601SNOW 0


## Created values by separating string from digits

In [359]:
DataObtained['Value'] =DataObtained['value'].str[-4:]
DataObtained.head(20)

,year,month,element,id,Day,value,Value
0,2015,09,PRCP,US10buff039,1,US10buff039201509PRCP-9999,9999
1,2015,09,SNOW,US10buff039,1,US10buff039201509SNOW-9999,9999
2,2015,10,PRCP,US10buff039,1,US10buff039201510PRCP 0,0
3,2015,10,SNOW,US10buff039,1,US10buff039201510SNOW-9999,9999
4,2015,11,PRCP,US10buff039,1,US10buff039201511PRCP 0,0
5,2015,11,SNOW,US10buff039,1,US10buff039201511SNOW 0,0
6,2015,12,PRCP,US10buff039,1,US10buff039201512PRCP 13,13
7,2015,12,SNOW,US10buff039,1,US10buff039201512SNOW 0,0
8,2016,01,PRCP,US10buff039,1,US10buff039201601PRCP 0,0
9,2016,01,SNOW,US10buff039,1,US10buff039201601SNOW 0,0


In [360]:
datclean=DataObtained[['year','month','Day','id','element','Value']]
datclean.head(10)

,year,month,Day,id,element,Value
0,2015,09,1,US10buff039,PRCP,9999
1,2015,09,1,US10buff039,SNOW,9999
2,2015,10,1,US10buff039,PRCP,0
3,2015,10,1,US10buff039,SNOW,9999
4,2015,11,1,US10buff039,PRCP,0
5,2015,11,1,US10buff039,SNOW,0
6,2015,12,1,US10buff039,PRCP,13
7,2015,12,1,US10buff039,SNOW,0
8,2016,01,1,US10buff039,PRCP,0
9,2016,01,1,US10buff039,SNOW,0


## Converted Values to Numerical Values

In [367]:
xy=datclean
xy["Value"]=xy[["Value"]].apply(pd.to_numeric, errors='coerce',axis=1)
xy=xy.replace(9999.0,np.nan)

xy.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,year,month,Day,id,element,Value
0,2015,09,1,US10buff039,PRCP,NaN
1,2015,09,1,US10buff039,SNOW,NaN
2,2015,10,1,US10buff039,PRCP,0.0
3,2015,10,1,US10buff039,SNOW,NaN
4,2015,11,1,US10buff039,PRCP,0.0
5,2015,11,1,US10buff039,SNOW,0.0
6,2015,12,1,US10buff039,PRCP,13.0
7,2015,12,1,US10buff039,SNOW,0.0
8,2016,01,1,US10buff039,PRCP,0.0
9,2016,01,1,US10buff039,SNOW,0.0


# Droped Days with no readings using dropna()

In [580]:
df2=xy

df2=df2.dropna(subset=['Value'])
df2.head()

,year,month,Day,id,element,Value
2,2015,10,1,US10buff039,PRCP,0.0
4,2015,11,1,US10buff039,PRCP,0.0
5,2015,11,1,US10buff039,SNOW,0.0
6,2015,12,1,US10buff039,PRCP,13.0
7,2015,12,1,US10buff039,SNOW,0.0


## Spread Data 

In [373]:
# xy.pivot(index=['year','month','Day','id'], columns='element',values='Value')
xyz =df2.pivot_table(index=['id','year','month','Day'],columns='element',values='Value')
xyz=xyz.reset_index()
xyz.head()

element,id,year,month,Day,4986,AWND,DAPR,DASF,EVAP,MDPR,...,WT03,WT04,WT05,WT06,WT08,WT09,WT11,WT14,WT16,WT18
0,US10buff039,2015,09,25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US10buff039,2015,09,26,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US10buff039,2015,09,27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US10buff039,2015,09,28,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US10buff039,2015,09,29,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Concat day year month to datestring

In [374]:

xyz['Date']  = xyz['Day'].map(str) + '-' + xyz['month'].map(str) + '-' + xyz['year'].map(str)
xyz.head()

element,id,year,month,Day,4986,AWND,DAPR,DASF,EVAP,MDPR,...,WT04,WT05,WT06,WT08,WT09,WT11,WT14,WT16,WT18,Date
0,US10buff039,2015,09,25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25-09-2015
1,US10buff039,2015,09,26,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26-09-2015
2,US10buff039,2015,09,27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27-09-2015
3,US10buff039,2015,09,28,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28-09-2015
4,US10buff039,2015,09,29,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29-09-2015


In [375]:
xyz.columns

Index(['id', 'year', 'month', 'Day', '4986', 'AWND', 'DAPR', 'DASF', 'EVAP',
       'MDPR', 'MDSF', 'MNPN', 'MXPN', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN',
       'TOBS', 'WDFG', 'WDMV', 'WESD', 'WESF', 'WSFG', 'WT01', 'WT03', 'WT04',
       'WT05', 'WT06', 'WT08', 'WT09', 'WT11', 'WT14', 'WT16', 'WT18', 'Date'],
      dtype='object', name='element')

## selected prefered elements

In [581]:
df3=xyz[['Date','year', 'month', 'Day','id' ,'TMAX', 'TMIN','PRCP']]

df3.head()

element,Date,year,month,Day,id,TMAX,TMIN,PRCP
0,25-09-2015,2015,09,25,US10buff039,NaN,NaN,0.0
1,26-09-2015,2015,09,26,US10buff039,NaN,NaN,0.0
2,27-09-2015,2015,09,27,US10buff039,NaN,NaN,NaN
3,28-09-2015,2015,09,28,US10buff039,NaN,NaN,0.0
4,29-09-2015,2015,09,29,US10buff039,NaN,NaN,0.0


# dropped days with missing TMAX and TMIN

In [582]:
df3=df3.dropna(subset=['TMAX', 'TMIN'])
df3.head()

element,Date,year,month,Day,id,TMAX,TMIN,PRCP
2564695,19-04-2018,2018,04,19,USC00013931,289.0,167.0,0.0
2564696,20-04-2018,2018,04,20,USC00013931,228.0,61.0,0.0
2564697,21-04-2018,2018,04,21,USC00013931,211.0,61.0,0.0
2564698,22-04-2018,2018,04,22,USC00013931,239.0,83.0,0.0
2564699,23-04-2018,2018,04,23,USC00013931,206.0,144.0,185.0


In [500]:
df3.describe()

element,Day,TMAX,TMIN,PRCP
count,44054.000000,44054.000000,44054.000000,40312.000000
mean,15.719163,177.560290,56.500704,28.953339
std,8.815408,119.984415,106.599556,86.811489
min,1.000000,-283.000000,-728.000000,0.000000
25%,8.000000,83.000000,-11.000000,0.000000
50%,16.000000,194.500000,61.000000,0.000000
75%,23.000000,278.000000,139.000000,10.000000
max,31.000000,999.000000,999.000000,1529.000000


# Converted TMAX and TMIN from tens of degree celsius to fahrenheit

In [463]:
Final_Data=df3[['Date','year', 'month', 'Day','id' ,'TMAX', 'TMIN','PRCP']]
Final_Data['TMAX']=((Final_Data['TMAX']/10)*(9/5))+32
Final_Data['TMIN']=((Final_Data['TMIN']/10)*(9/5))+32

In [501]:
Final_Data.describe()

element,Day,TMAX,TMIN,PRCP
count,44054.000000,44054.000000,44054.000000,40312.000000
mean,15.719163,63.960852,42.170127,28.953339
std,8.815408,21.597195,19.187920,86.811489
min,1.000000,-18.940000,-99.040000,0.000000
25%,8.000000,46.940000,30.020000,0.000000
50%,16.000000,67.010000,42.980000,0.000000
75%,23.000000,82.040000,57.020000,10.000000
max,31.000000,211.820000,211.820000,1529.000000


# Converted datestring to datetime objects

In [583]:
dt=Final_Data
dt['Date'] = dt["Date"].apply(pd.to_datetime,format='%d-%m-%Y', errors='coerce')
dt.head()

element,Date,year,month,Day,id,TMAX,TMIN,PRCP
2564695,2018-04-19,2018,04,19,USC00013931,84.02,62.06,0.0
2564696,2018-04-20,2018,04,20,USC00013931,73.04,42.98,0.0
2564697,2018-04-21,2018,04,21,USC00013931,69.98,42.98,0.0
2564698,2018-04-22,2018,04,22,USC00013931,75.02,46.94,0.0
2564699,2018-04-23,2018,04,23,USC00013931,69.08,57.92,185.0


# Used pandas data reader to obtain stock info for the 3 companies(i.e Mosaic company, CF Industries, and FMC Corp) from 2015 to date.

In [465]:
tickers = ['MOS', 'CF','FMC']
today = str(date.today())

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2015-01-01'
end_date = today

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = data.DataReader(tickers , 'yahoo', start_date, end_date)

# Selected adjusted close prices for the 3 companies.

In [499]:
adj_close=panel_data.loc[:,'Adj Close']
adj_close.reset_index()
adj_close.head()

Symbols,CF,FMC,MOS
Date,,,
2015-01-02,46.257175,54.539059,41.682686
2015-01-05,46.099045,53.342392,40.935921
2015-01-06,47.052822,53.687031,41.482330
2015-01-07,48.394428,54.089108,41.700901
2015-01-08,49.066891,55.429367,42.192677


# Joined stock data and weather data on date

In [584]:
Final_Data2=pd.merge(dt,adj_close,how='left',left_on='Date',right_on='Date')
Final_Data2.head()

,Date,year,month,Day,id,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2018-04-19,2018,04,19,USC00013931,84.02,62.06,0.0,38.858654,81.815094,25.805868
1,2018-04-20,2018,04,20,USC00013931,73.04,42.98,0.0,37.956787,82.004379,25.636158
2,2018-04-21,2018,04,21,USC00013931,69.98,42.98,0.0,NaN,NaN,NaN
3,2018-04-22,2018,04,22,USC00013931,75.02,46.94,0.0,NaN,NaN,NaN
4,2018-04-23,2018,04,23,USC00013931,69.08,57.92,185.0,38.231266,82.094048,25.845800


# Filled forward the stock prices to cover the weekends.

In [586]:

Final_Data2.fillna( method ='ffill', inplace = True) 

In [587]:
Final_Data2.head()

,Date,year,month,Day,id,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2018-04-19,2018,04,19,USC00013931,84.02,62.06,0.0,38.858654,81.815094,25.805868
1,2018-04-20,2018,04,20,USC00013931,73.04,42.98,0.0,37.956787,82.004379,25.636158
2,2018-04-21,2018,04,21,USC00013931,69.98,42.98,0.0,37.956787,82.004379,25.636158
3,2018-04-22,2018,04,22,USC00013931,75.02,46.94,0.0,37.956787,82.004379,25.636158
4,2018-04-23,2018,04,23,USC00013931,69.08,57.92,185.0,38.231266,82.094048,25.845800


# Sorted dates in ascending order

In [588]:


Final_Data2.sort_values(by='Date',ascending=True,inplace=True)

In [589]:
Final_Data2.reset_index()
Final_Data2.head()

,Date,year,month,Day,id,TMAX,TMIN,PRCP,CF,FMC,MOS
6247,2015-01-01,2015,01,1,USC00084621,73.94,60.98,0.0,26.720013,72.355659,22.570574
6248,2015-01-02,2015,01,2,USC00084621,80.06,62.96,0.0,46.257175,54.539059,41.682686
6249,2015-01-03,2015,01,3,USC00084621,80.06,64.04,0.0,46.257175,54.539059,41.682686
6250,2015-01-04,2015,01,4,USC00084621,84.02,64.94,0.0,46.257175,54.539059,41.682686
6251,2015-01-05,2015,01,5,USC00084621,84.92,62.96,0.0,46.099045,53.342392,40.935921


In [554]:
Final_Data2.columns

Index(['Date', 'year', 'month', 'Day', 'id', 'TMAX', 'TMIN', 'PRCP', 'CF',
       'FMC', 'MOS'],
      dtype='object')

In [555]:
Preffered_stns.columns

Index(['ID', 'LAT', 'LON', 'ELEV', 'ST', 'NAME', 'GSN', 'HCN', 'WMOID', 'ELEM',
       'FIRST', 'LAST', 'Country_code'],
      dtype='object')

# Removed duplicates from preferred stations based on station ID

In [556]:
Preffered_stns.drop_duplicates(subset='ID',inplace=True)

# Merged data with preferred stations to obtain location, name, elevation, and state of station.

In [591]:
Final_Data3=pd.merge(Final_Data2,Preffered_stns[['ID', 'LAT', 'LON','ST','ELEV', 'NAME']],how='left',left_on='id',\
                     right_on='ID')

In [592]:
Final_Data3.head()

,Date,year,month,Day,id,TMAX,TMIN,PRCP,CF,FMC,MOS,ID,LAT,LON,ST,ELEV,NAME
0,2015-01-01,2015,01,1,USC00084621,73.94,60.98,0.0,26.720013,72.355659,22.570574,USC00084621,27.6039,-81.0531,FL,21.9,KISSIMMEE
1,2015-01-02,2015,01,2,USC00084621,80.06,62.96,0.0,46.257175,54.539059,41.682686,USC00084621,27.6039,-81.0531,FL,21.9,KISSIMMEE
2,2015-01-03,2015,01,3,USC00084621,80.06,64.04,0.0,46.257175,54.539059,41.682686,USC00084621,27.6039,-81.0531,FL,21.9,KISSIMMEE
3,2015-01-04,2015,01,4,USC00084621,84.02,64.94,0.0,46.257175,54.539059,41.682686,USC00084621,27.6039,-81.0531,FL,21.9,KISSIMMEE
4,2015-01-05,2015,01,5,USC00084621,84.92,62.96,0.0,46.099045,53.342392,40.935921,USC00084621,27.6039,-81.0531,FL,21.9,KISSIMMEE


In [559]:
Final_Data3.columns

Index(['Date', 'year', 'month', 'Day', 'id', 'TMAX', 'TMIN', 'PRCP', 'CF',
       'FMC', 'MOS', 'ID', 'LAT', 'LON', 'ST', 'ELEV', 'NAME'],
      dtype='object')

In [594]:
Data_To_Load=Final_Data3[['Date', 'year', 'month','Day','ID','NAME','ELEV','ST', 'LAT', 'LON','TMAX', 'TMIN', 'PRCP', 'CF',
       'FMC', 'MOS']]

In [595]:
Data_To_Load.head()

,Date,year,month,Day,ID,NAME,ELEV,ST,LAT,LON,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2015-01-01,2015,01,1,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,73.94,60.98,0.0,26.720013,72.355659,22.570574
1,2015-01-02,2015,01,2,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,62.96,0.0,46.257175,54.539059,41.682686
2,2015-01-03,2015,01,3,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,64.04,0.0,46.257175,54.539059,41.682686
3,2015-01-04,2015,01,4,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.02,64.94,0.0,46.257175,54.539059,41.682686
4,2015-01-05,2015,01,5,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.92,62.96,0.0,46.099045,53.342392,40.935921


# Replace outliers with numpy missing values .

In [596]:
Data_To_Load['TMAX'] = Data_To_Load['TMAX'].mask(Data_To_Load['TMAX'] > 120, np.nan)
Data_To_Load['TMIN'] = Data_To_Load['TMIN'].mask(Data_To_Load['TMIN'] > 120, np.nan)
Data_To_Load['TMIN'] = Data_To_Load['TMIN'].mask(Data_To_Load['TMIN'] <- 50, np.nan)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [597]:
Data_To_Load.head()

,Date,year,month,Day,ID,NAME,ELEV,ST,LAT,LON,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2015-01-01,2015,01,1,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,73.94,60.98,0.0,26.720013,72.355659,22.570574
1,2015-01-02,2015,01,2,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,62.96,0.0,46.257175,54.539059,41.682686
2,2015-01-03,2015,01,3,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,64.04,0.0,46.257175,54.539059,41.682686
3,2015-01-04,2015,01,4,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.02,64.94,0.0,46.257175,54.539059,41.682686
4,2015-01-05,2015,01,5,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.92,62.96,0.0,46.099045,53.342392,40.935921


# Filled forward to mask outliers with previous day's reading

In [598]:
Data_To_Load.fillna( method ='ffill', inplace = True) 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [599]:
Data_To_Load.head()

,Date,year,month,Day,ID,NAME,ELEV,ST,LAT,LON,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2015-01-01,2015,01,1,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,73.94,60.98,0.0,26.720013,72.355659,22.570574
1,2015-01-02,2015,01,2,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,62.96,0.0,46.257175,54.539059,41.682686
2,2015-01-03,2015,01,3,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,64.04,0.0,46.257175,54.539059,41.682686
3,2015-01-04,2015,01,4,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.02,64.94,0.0,46.257175,54.539059,41.682686
4,2015-01-05,2015,01,5,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.92,62.96,0.0,46.099045,53.342392,40.935921


In [566]:
Data_To_Load.describe()

,Day,ELEV,LAT,LON,TMAX,TMIN,PRCP,CF,FMC,MOS
count,44054.000000,44054.000000,44054.000000,44054.000000,44054.000000,44054.000000,44054.000000,44054.000000,44054.000000,44054.000000
mean,15.719163,625.905813,39.748654,-90.374033,63.879204,42.080106,32.019839,36.487127,74.578924,26.927745
std,8.815408,673.754718,4.970374,13.964649,21.325050,18.769062,90.339744,8.961287,17.554919,4.328270
min,1.000000,0.900000,25.916100,-117.038100,-18.940000,-36.040000,0.000000,19.369413,31.865461,19.238867
25%,8.000000,173.100000,35.721400,-103.554500,46.940000,30.020000,0.000000,28.027294,58.429249,23.765696
50%,16.000000,340.200000,40.821700,-86.248600,66.920000,42.980000,0.000000,36.599506,81.947739,26.789148
75%,23.000000,876.600000,43.017800,-81.890600,82.040000,57.020000,13.000000,42.356483,88.355904,29.226280
max,31.000000,3161.400000,48.116700,-69.964700,114.080000,84.020000,1529.000000,60.328136,97.591125,48.777031


In [567]:
Data_To_Load.head(10)


,Date,year,month,Day,ID,NAME,ELEV,ST,LAT,LON,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2015-01-01,2015,01,1,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,73.94,60.98,0.0,26.720013,72.355659,22.570574
1,2015-01-02,2015,01,2,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,62.96,0.0,46.257175,54.539059,41.682686
2,2015-01-03,2015,01,3,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,64.04,0.0,46.257175,54.539059,41.682686
3,2015-01-04,2015,01,4,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.02,64.94,0.0,46.257175,54.539059,41.682686
4,2015-01-05,2015,01,5,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.92,62.96,0.0,46.099045,53.342392,40.935921
5,2015-01-06,2015,01,6,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,75.02,53.96,0.0,47.052822,53.687031,41.482330
6,2015-01-07,2015,01,7,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.96,51.08,0.0,48.394428,54.089108,41.700901
7,2015-01-08,2015,01,8,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,71.06,39.92,0.0,49.066891,55.429367,42.192677
8,2015-01-09,2015,01,9,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,64.94,46.04,0.0,49.466377,55.084732,41.855717
9,2015-01-10,2015,01,10,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,77.00,46.94,0.0,49.466377,55.084732,41.855717


## Set Index to dates

In [600]:
Data_To_Load.set_index("Date", inplace=True)

## Investigate Data Types

In [601]:
Data_To_Load.dtypes

year      object
month     object
Day        int64
ID        object
NAME      object
ELEV     float64
ST        object
LAT      float64
LON      float64
TMAX     float64
TMIN     float64
PRCP     float64
CF       float64
FMC      float64
MOS      float64
dtype: object

# Prepare mysql connection to load data

In [647]:
import pymysql
pymysql.install_as_MySQLdb()
connection_string = "root:toor@localhost/ETL_Project"
engine = create_engine(f'mysql://{connection_string}')

engine.table_names()

['weather_agric_stocks', 'weather_agric_stocks2']

In [648]:
engine.execute("DROP TABLE IF EXISTS Weather_Agric_Stocks")

In [650]:
engine.execute("CREATE TABLE Weather_Agric_Stocks (Date VARCHAR(20) PRIMARY KEY ,\
    year VARCHAR(4) ,month VARCHAR(2) ,day VARCHAR(2) , ID varchar(11)  , NAME VARCHAR(67) , ELEV float4,\
    ST  VARCHAR(2) ,TMAX float4,TMIN float4,PRCP float4,LON float4,LAT float4,FMC float4,CF float4,MOS float4)")

In [651]:
engine.table_names()

['weather_agric_stocks', 'weather_agric_stocks2']

## LOAD DATA

In [654]:
Data_To_Load.to_sql(name='weather_agric_stocks', con=engine, if_exists='replace', index=True,chunksize=100)

# Confirm By quering the Weather Agric Stock Table

In [655]:
pd.read_sql_query('select * from weather_agric_stocks',con=engine).head(10)

,Date,year,month,Day,ID,NAME,ELEV,ST,LAT,LON,TMAX,TMIN,PRCP,CF,FMC,MOS
0,2015-01-01,2015,01,1,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,73.94,60.98,0.0,26.720013,72.355659,22.570574
1,2015-01-02,2015,01,2,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,62.96,0.0,46.257175,54.539059,41.682686
2,2015-01-03,2015,01,3,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.06,64.04,0.0,46.257175,54.539059,41.682686
3,2015-01-04,2015,01,4,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.02,64.94,0.0,46.257175,54.539059,41.682686
4,2015-01-05,2015,01,5,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,84.92,62.96,0.0,46.099045,53.342392,40.935921
5,2015-01-06,2015,01,6,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,75.02,53.96,0.0,47.052822,53.687031,41.482330
6,2015-01-07,2015,01,7,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,80.96,51.08,0.0,48.394428,54.089108,41.700901
7,2015-01-08,2015,01,8,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,71.06,39.92,0.0,49.066891,55.429367,42.192677
8,2015-01-09,2015,01,9,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,64.94,46.04,0.0,49.466377,55.084732,41.855717
9,2015-01-10,2015,01,10,USC00084621,KISSIMMEE,21.9,FL,27.6039,-81.0531,77.00,46.94,0.0,49.466377,55.084732,41.855717
